In [130]:
import pickle

from mlscorecheck.utils import ProblemSolver, generate_problem, generate_problem_tp0
from mlscorecheck.core import score_function_set, sqrt
from mlscorecheck.core import Interval, union, reduce_solutions

import sympy

In [131]:
scores = score_function_set()

In [132]:
score0 = 'ppv'
score1 = 'ji'

score0_fun = scores[score0]
score1_fun = scores[score1]

In [133]:
tp = sympy.Symbol('tp')
tn = sympy.Symbol('tn')
fp = sympy.Symbol('fp')
fn = sympy.Symbol('fn')
p = sympy.Symbol('p')
n = sympy.Symbol('n')

symbols = {'tp': tp,
            'tn': tn,
            'fp': fp,
            'fn': fn,
            'p': p,
            'n': n}

score0_sym = sympy.Symbol(score0)
score1_sym = sympy.Symbol(score1)

In [134]:
score0_args = score0_fun.__code__.co_varnames[:score0_fun.__code__.co_kwonlyargcount]
score1_args = score1_fun.__code__.co_varnames[:score1_fun.__code__.co_kwonlyargcount]

In [135]:
score0_args, score1_args

(('tp', 'fp'), ('tp', 'fp', 'p'))

In [136]:
score0_exp = score0_fun(**{key: val for key, val in symbols.items() if key in score0_args})
score1_exp = score1_fun(**{key: val for key, val in symbols.items() if key in score1_args})
score0_eq = score0_sym - score0_exp
score1_eq = score1_sym - score1_exp
n_eq = n - tn - fp
p_eq = p - tp - fn

In [137]:
score0_exp, score1_exp

(tp/(fp + tp), tp/(fp + p))

In [138]:
num0, denom0 = score0_exp.as_numer_denom()
num1, denom1 = score1_exp.as_numer_denom()

In [139]:
num0, denom0, num1, denom1

(tp, fp + tp, tp, fp + p)

In [140]:
score0_eq = sympy.simplify(score0_eq * denom0)
score1_eq = sympy.simplify(score1_eq * denom1)

In [141]:
score0_eq, score1_eq, n_eq, p_eq

(ppv*(fp + tp) - tp, ji*(fp + p) - tp, -fp + n - tn, -fn + p - tp)

In [146]:
gb = sympy.GroebnerBasis([score0_eq, score1_eq, n_eq, p_eq], tp, tn, fn, fp, order='grevlex')

In [147]:
gb

GroebnerBasis([-ji*p*ppv + tp*(ji*ppv - ji + ppv), -ji*n*ppv - ji*p*ppv + ji*n + ji*p - n*ppv + tn*(ji*ppv - ji + ppv), ji*p - p*ppv + fn*(ji*ppv - ji + ppv), ji*p*ppv - ji*p + fp*(ji*ppv - ji + ppv)], tp, tn, fn, fp, domain='ZZ[p,n,ji,ppv]', order='grevlex')

In [152]:
gb[0]

-ji*p*ppv + tp*(ji*ppv - ji + ppv)

In [155]:
gb[0].coeff(tp)

ji*ppv - ji + ppv

In [ ]:
#ji*ppv - ji + ppv = 0
#ji*ppv = ji - ppv

In [ ]:
ji*ppv - ji + ppv = 

In [145]:
sympy.solve(score1_exp * score0_exp - score1_exp + score0_exp)

[{p: 0}, {tp: 0}]

In [92]:
sympy.GroebnerBasis([npv_eq, n_eq, p_eq], tp, tn, fn, fp, order='lex')

GroebnerBasis([fp*(npv - 1) - n*npv + n - npv*p + npv*tp, fp - n + tn, fn*npv + fp*(1 - npv) + n*npv - n], tp, tn, fn, fp, domain='ZZ[p,n,npv]', order='lex')

In [93]:
sympy.GroebnerBasis([spec_eq, npv_eq, n_eq, p_eq], tp, tn, fn, fp, order='lex').is_zero_dimensional

True

In [94]:
subst = {npv: 0}
spec_eq_new = spec_eq.subs(subst)
npv_eq_new = npv_eq.subs(subst)
n_eq_new = n_eq.subs(subst)
p_eq_new = p_eq.subs(subst)

In [95]:
sympy.GroebnerBasis([spec_eq_new, npv_eq_new, n_eq_new, p_eq_new], tp, tn, fn, fp, order='lex')

GroebnerBasis([1], tp, tn, fn, fp, domain='ZZ[p,n,spec]', order='lex')

In [76]:
sympy.GroebnerBasis([spec_eq_new, npv_eq_new, n_eq_new, p_eq_new], tp, tn, fn, fp, order='lex').is_zero_dimensional

False

In [77]:
spec_eq_new, npv_eq_new, n_eq_new, p_eq_new

((n + p)*(n*spec - tn)/n, -tn, -fp + n - tn, -fn + p - tp)

In [78]:
sympy.GroebnerBasis([acc_eq_new, npv_eq_new, n_eq_new, p_eq_new], tp, tn, fn, fp, order='lex')

GroebnerBasis([-acc*n - acc*p + tp, tn, acc*n + acc*p + fn - p, fp - n], tp, tn, fn, fp, domain='ZZ[p,n,acc]', order='lex')

In [79]:
sympy.solve_poly_system([acc_eq, npv_eq, n_eq, p_eq], tp, tn, fp, fn)

[((acc*n*npv - acc*n + acc*npv*p - acc*p + npv*p)/(2*npv - 1),
  npv*(acc*n + acc*p - p)/(2*npv - 1),
  -(acc*n*npv + acc*npv*p - 2*n*npv + n - npv*p)/(2*npv - 1),
  -(npv - 1)*(acc*n + acc*p - p)/(2*npv - 1))]